# 13F Holdings Analysis: AI Stocks

This notebook analyzes institutional investor behavior in AI stocks through SEC 13F filings.

**Key questions:**
- Which hedge funds have the largest holdings in AI stocks?
- How has institutional ownership of AI stocks evolved over time?
- Which specific AI stocks do top funds favor?

## Setup

In [ ]:
try:
    import pandas as pd
    from pathlib import Path
    from ai_stocks import AI_STOCKS, ALL_AI_TICKERS
    import json
    print('Packages and dependencies loaded successfully.')
except ImportError as e:
    print(f'Failed to load package or dependency: {e}')

## Load 13F Data

SEC 13F filings are distributed as bulk TSV files. Quarters are stored in directories like `data/raw/2023q4_form13f/` with three key files:
- `INFOTABLE.tsv` - holdings (CUSIP, value, shares)
- `COVERPAGE.tsv` - filer metadata (fund name, filing date)
- `SUBMISSION.tsv` - submission data (type of filing, relevant period)

First, we build a ticker lookup dictionary using the provided JSON file. From this, we create a lookup dictionary specifically for AI stock tickers.

In [ ]:
DATA_DIR = Path("data/raw")

with open('data/reference/company-tickers.json', 'r') as ticker_map_file:
    sec_tickers = json.load(ticker_map_file)

    # the file is stored as a dictionary of numbers to mappings: 
    # {"0":{"cik_str":1045810,"ticker":"NVDA","title":"NVIDIA CORP"},"1":{"cik_str":320193,"ticker":"AAPL"}, ...}

    # build a dictionary from the company name to the ticker symbol
    name_to_ticker = {entry['title'].upper(): entry['ticker'] for entry in sec_tickers.values()}

# crucial for ticker matching that occurs in the later stages: reduce matching from ~10,000 entries to ~50
ai_name_to_ticker = {name: ticker for name, ticker in name_to_ticker.items() if ticker in ALL_AI_TICKERS}

# build reverse mapping: ticker -> AI category
ticker_to_ai_type = {ticker: category for category, tickers in AI_STOCKS.items() for ticker in tickers}

ai_name_to_ticker

In [ ]:
def list_quarters():
    """List available quarters of 13F data."""
    # extract the name of each directory
    quarters = [d.name for d in DATA_DIR.iterdir() if d.is_dir()]
    
    return sorted(quarters)

print('Available quarters:', list_quarters())

## Loading quarter data
It is natural to analyze the data in a quarterly fashion.  
We will only read in parts of the data relevant to our analysis. In particular:
- INFOTABLE.tsv: `ACCESSION_NUMBER`, `INFOTABLE_SK`, `NAMEOFISSUER`, `CUSIP`, `FIGI`, `VALUE`, `SSHPRNAMT`, `SSHPRNAMTTYPE`, `PUTCALL`
- COVERPAGE.tsv: `ACCESSION_NUMBER`, `REPORTCALENDARORQUARTER`, `FILINGMANAGER_NAME`
- SUBMISSION.tsv: `ACCESSION_NUMBER`, `FILING_DATE`, `SUBMISSION_TYPE`, `PERIODOFREPORT`

There is an important _pre-processing step_ that occurs here. The SEC made more stringent reporting standards for Form 13F effective 1/3/2023.  
In particular, the dollar values before that date were reported to the closest thousand, whereas after that date they are reported to the closest dollar.

To make appropriate comparisons between quarters and understand the dollar values, we must normalize the pre-2023 dollar amounts by multiplying the raw amounts by 1000.

In [ ]:
# Mapping from quarter string to expected PERIODOFREPORT date
QUARTER_TO_PERIOD_PRE2024 = {
    'q1': '-12-31',
    'q2': '-03-31',
    'q3': '-06-30',
    'q4': '-09-30'
}

def exp_period(quarter: str) -> pd.Timestamp:
    """Get the expected period of report date from a string representing year/quarter."""
    # Parse quarter string (e.g., '2023q4') to get expected period date
    year = int(quarter[:4]) # e.g., 2023
    q = quarter[4:]    # e.g., 'q4'

    if year < 2024:
        # if quarter is q1, expected period of filing lies in previous year
        if q == 'q1':
            year -= 1
            
        return pd.Timestamp(f'{year}-{QUARTER_TO_PERIOD_PRE2024[q]}')
    else:
        # TODO: adapt to parse this data
        print('Error: parsing quarterly data from 2024 onwards is unsupported due to a change in the filing dates.')
        raise ValueError

    

def load_quarter(quarter: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Load holdings, filing, and amendment data for a specific quarter.
    
    Returns:
        holdings_reports: 13F-HR filings for the quarter's period
        filings: COVERPAGE data
        amendment_reports: 13F-HR/A filings for the quarter's period
    """
    quarter_dir = DATA_DIR / quarter

    expected_period = exp_period(quarter)
    
    infotable_datatypes = {
        'ACCESSION_NUMBER': 'string',
        'INFOTABLE_SK': 'Int64',
        'NAMEOFISSUER': 'string',
        'CUSIP': 'string',
        'FIGI': 'string',
        'VALUE': 'Int64',
        'SSHPRNAMT': 'Int64',
        'SSHPRNAMTTYPE': 'string',
        'PUTCALL': 'string',
    }

    filings_datatypes = {
        'ACCESSION_NUMBER': 'string',
        'REPORTCALENDARORQUARTER': 'string',
        'FILINGMANAGER_NAME': 'string'
    }

    submissions_datatypes = {
        'ACCESSION_NUMBER': 'string',
        'FILING_DATE': 'string',
        'SUBMISSIONTYPE': 'string',
        'PERIODOFREPORT': 'string'
    }

    # read tables and rename columns for readability

    holdings = pd.read_table(
        quarter_dir / 'INFOTABLE.tsv',  
        dtype=infotable_datatypes,
        usecols=infotable_datatypes.keys()
    ).rename(columns={'SSHPRNAMT': 'SHARES/PRINCIPAL AMOUNT', 'SSHPRNAMTTYPE': 'AMOUNT_UNIT', 'NAMEOFISSUER': 'COMPANY_NAME'})

    filings = pd.read_table(
        quarter_dir / 'COVERPAGE.tsv',
        dtype=filings_datatypes,
        usecols=filings_datatypes.keys(),
        parse_dates=['REPORTCALENDARORQUARTER'],
        date_format='%d-%b-%Y'
    ).rename(columns={'FILINGMANAGER_NAME': 'MANAGER_NAME'})

    raw_submissions = pd.read_table(
        quarter_dir / 'SUBMISSION.tsv',
        dtype=submissions_datatypes,
        usecols=submissions_datatypes.keys(),
        parse_dates=['FILING_DATE', 'PERIODOFREPORT'],
        date_format='%d-%b-%Y'
    )

    # Filter to 13F-HR and 13F-HR/A submissions for the expected period only
    relevant_submissions = raw_submissions[
        (raw_submissions['SUBMISSIONTYPE'].isin(['13F-HR', '13F-HR/A'])) &
        (raw_submissions['PERIODOFREPORT'] == expected_period)
    ]

    holdings = pd.merge(holdings, relevant_submissions, on=['ACCESSION_NUMBER'])

    # Normalize VALUE: pre-2023 data reported in thousands, 2023+ in dollars
    VALUE_CUTOFF_DATE = pd.Timestamp('2023-01-03')
    pre_2023_mask = holdings['FILING_DATE'] < VALUE_CUTOFF_DATE
    holdings.loc[pre_2023_mask, 'VALUE'] = holdings.loc[pre_2023_mask, 'VALUE'] * 1000

    holdings['QUARTER'] = quarter

    # Split into holding reports and amendments
    is_amendment = holdings['SUBMISSIONTYPE'] == '13F-HR/A'

    holdings_reports = holdings[~is_amendment]
    amendment_reports = holdings[is_amendment]
    
    return holdings_reports, filings, amendment_reports


# Load most recent quarter
quarters = list_quarters()
sample_holdings, sample_filings, sample_amendments = load_quarter(quarters[-1])
print(f'Loaded {len(sample_holdings):,} holdings and {len(sample_amendments):,} amendments from {len(sample_filings):,} filers')

### Inspect the data

In [ ]:
sample_holdings.head().style.format(thousands=',', subset=['VALUE'])

In [ ]:
sample_filings.head()

### Diagnostics: Data Quality & Filing Integrity

SEC bulk 13F datasets for a given quarter are not limited to a single, clean snapshot of holdings. They often include amendments, late filings, and other administrative records that can contaminate the true quarter-end view. Before running any financial or time-series analysis, we need to validate that the dataset represents what we think it does.

In particular, we check for three common failure modes:

- Cross-period contamination: verifying that holdings from other PERIODOFREPORT dates are not leaking into the quarter being analyzed.

- Duplicate manager–security pairs: ensuring that each filer–stock combination appears only once after filtering, so aggregate values are not artificially inflated.

- Non-holdings filings: confirming that we are only including true holdings filings (e.g., not notices, amendments, or other submission types that should not contribute to portfolio value).

These diagnostics act as a guardrail: if any of them fail, the downstream valuation, aggregation, and trend analyses cannot be trusted.

In [ ]:
# Diagnostic 1: Check PERIODOFREPORT distribution
# For instance, the 2023q4 bulk file should only contain filings for 2023-09-30
print('=== PERIODOFREPORT Distribution ===')
period_counts = sample_holdings['PERIODOFREPORT'].value_counts()
print(period_counts.head(15))

if len(period_counts) != 1:
    print('error: holdings contained more than one date')
    raise ValueError

print(f'\nDate: {period_counts.index[0].date()} in {period_counts.iloc[0]:,} rows')

In [ ]:
# Diagnostic 2: Check for duplicate manager/stock combinations within the quarter
# Join with filings to get manager name, then group by manager + stock
merged_holdings = pd.merge(sample_holdings, sample_filings, on='ACCESSION_NUMBER')
dupes = merged_holdings.groupby(['MANAGER_NAME', 'COMPANY_NAME']).agg(
    NUM_ACCESSIONS=('ACCESSION_NUMBER', 'nunique'),
    TOTAL_VALUE=('VALUE', 'sum'),
    NUM_ROWS=('VALUE', 'count')
).reset_index()

# Filter to cases with more than one row per manager/stock
multi_row = dupes[dupes['NUM_ROWS'] > 1].copy().sort_values('TOTAL_VALUE', ascending=False)
print(f'=== Manager/Stock combinations with multiple rows ===')
print(f'Total combinations with >1 row: {len(multi_row):,}')
print(f'\nTop 10 by total value:')
multi_row.head(10).style.format(thousands=',', subset=['TOTAL_VALUE'])

In [ ]:
# Diagnostic 3: Check SUBMISSIONTYPE distribution

print('=== SUBMISSIONTYPE Distribution ===')
submission_type_counts = sample_holdings['SUBMISSIONTYPE'].value_counts()
print(submission_type_counts.head(15))

if len(submission_type_counts) != 1:
    print('error: holdings contained more than one submission type')
    raise ValueError

submission_type = submission_type_counts.index[0]
if submission_type != '13F-HR':
    print(f'error: holdings contained non-holdings submission type: {submission_type}')
    raise ValueError

print(f'\nSubmission type: {submission_type_counts.index[0]} in {submission_type_counts.iloc[0]:,} rows')


In [ ]:
# Diagnostic 4: Verify one accession per manager/stock (holdings only, excluding amendments)
# Join holdings with filings to get manager names

# Group by manager + stock and count unique accessions
accession_check = merged_holdings.groupby(['MANAGER_NAME', 'COMPANY_NAME']).agg(
    NUM_ACCESSIONS=('ACCESSION_NUMBER', 'nunique'),
    TOTAL_VALUE=('VALUE', 'sum'),
    NUM_ROWS=('VALUE', 'count')
).reset_index()

# Check for any groups with multiple accessions
multi_accession = dupes[dupes['NUM_ACCESSIONS'] > 1]

print(f'=== Accession Uniqueness Check (Holdings Only) ===')
print(f'Total manager/stock groups: {len(dupes):,}')
print(f'Groups with NUM_ACCESSIONS > 1: {len(multi_accession):,}')

if len(multi_accession) > 0:
    print(f'\nTop 10 by value with multiple accessions:')
    print(multi_accession.sort_values('TOTAL_VALUE', ascending=False).head(10))
    raise ValueError('Found manager/stock groups with multiple accessions')
else:
    print('\nAll manager/stock groups have exactly one accession number associated with them.')

## Map Holdings to AI Stocks

13F filings identify securities by CUSIP. We'll match on issuer name to identify AI stocks.  
This is where the dictionary of AI stock tickers will prove useful.

In [ ]:
def match_ai_stock_to_ticker(issuer_name: str) -> str | None:
    """Map issuer name to stock ticker."""
    if pd.isna(issuer_name):
        return None
    
    issuer_upper = issuer_name.upper()
    
    # prioritize an exact match over a substring match
    if issuer_upper in ai_name_to_ticker:
        return ai_name_to_ticker[issuer_upper]
    
    for company_name, ticker in ai_name_to_ticker.items():
        if issuer_upper in company_name or company_name in issuer_upper:
            return ticker
    
    return None

Now, we can use the `filter_to_ai_stocks` function to:
1. Add an extra column to the DataFrame, corresponding to the stock ticker of the relevant AI stock or `None` if the stock indicated is not an AI stock
2. Given the augmented `sample_holdings` table, filter to only the entries with AI stocks

In [ ]:
def filter_to_ai_stocks(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds TICKER and AI_TYPE columns to the dataframe and returns a new 
    DataFrame representing only the entries related to AI stocks.
    """

    if 'COMPANY_NAME' not in df.columns.values:
        print('Error: company name not found in columns of dataframe.')
        raise LookupError
    
    df['TICKER'] = df['COMPANY_NAME'].apply(match_ai_stock_to_ticker)
    filtered = df.dropna(subset=['TICKER']).copy()
    filtered['AI_TYPE'] = filtered['TICKER'].map(ticker_to_ai_type)
    return filtered

# Apply matcher and filter
sample_ai_holdings = filter_to_ai_stocks(sample_holdings)

print(f'Found {len(sample_ai_holdings):,} filings related to AI holdings ({len(sample_ai_holdings)/len(sample_holdings)*100:.1f}% of total)')
ai_pct_value = sample_ai_holdings['VALUE'].sum()/sample_holdings['VALUE'].sum()
print(f'AI holdings are {ai_pct_value:.3f}% of total portfolio value')

sample_ai_holdings.head().style.format(thousands=',', subset=['VALUE'])

## Aggregate by Stock

We will first aggregate by stock to see which stocks have the highest value in terms of institutional investment value.

In [ ]:
# Total institutional holdings by AI stock
def aggregate_by_stock(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('TICKER').agg({'VALUE': 'sum', 'COMPANY_NAME': 'first', 'CUSIP': 'first'}).sort_values('VALUE', ascending=False)

agg_ai_by_stock = aggregate_by_stock(sample_ai_holdings)
agg_ai_by_stock.head(10).style.format(thousands=',', subset=['VALUE'])

## Aggregate holdings by fund
We can also understand which institutional investors are pursuing AI-related stocks the most. We will aggregate by filers.

In [ ]:
def aggregate_by_manager(holdings: pd.DataFrame, filings: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(holdings, filings, on=['ACCESSION_NUMBER']).groupby('MANAGER_NAME').agg({'VALUE': 'sum'}).sort_values('VALUE', ascending=False)

agg_ai_by_fund = aggregate_by_manager(sample_ai_holdings, sample_filings)
agg_ai_by_fund.head(15).style.format(thousands=',', subset=['VALUE'])

## Aggregate by AI Category
We can see which types of AI stocks are most popular with institutional investors.

In [ ]:
def aggregate_by_ai_type(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('AI_TYPE').agg({
        'VALUE': 'sum',
        'TICKER': 'nunique',
        'ACCESSION_NUMBER': 'nunique'
    }).rename(columns={'TICKER': 'NUM_STOCKS', 'ACCESSION_NUMBER': 'NUM_MANAGERS'}).sort_values('VALUE', ascending=False)

agg_by_type = aggregate_by_ai_type(sample_ai_holdings)
agg_by_type.style.format(thousands=',', subset=['VALUE'])

## Time Series Analysis
Load quarters from 2022 Q3 onward to analyze AI stock holdings trends during the AI boom.

In [ ]:
def load_quarters_from(start_quarter: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Load and concatenate quarters starting from start_quarter."""
    all_quarters = list_quarters()
    target_quarters = [q for q in all_quarters if q >= start_quarter]
    
    all_ai_holdings = []
    all_filings = []
    
    for q in target_quarters:
        holdings, filings, _ = load_quarter(q)
        ai_holdings = filter_to_ai_stocks(holdings)
        
        all_ai_holdings.append(ai_holdings)
        all_filings.append(filings)
        print(f'Loaded {q}: {len(ai_holdings):,} AI holdings')
    
    return pd.concat(all_ai_holdings, ignore_index=True), pd.concat(all_filings, ignore_index=True)

ai_holdings_ts, filings_ts = load_quarters_from('2022q3')
print(f'\nTotal: {len(ai_holdings_ts):,} AI holdings across {ai_holdings_ts["QUARTER"].nunique()} quarters')

### Holdings by AI Category Over Time

In [ ]:
ai_type_by_quarter = ai_holdings_ts.pivot_table(
    index='QUARTER', 
    columns='AI_TYPE', 
    values='VALUE', 
    aggfunc='sum',
    fill_value=0
)
ai_type_by_quarter.style.format(thousands=',')

### Top AI Stocks Over Time

In [ ]:
quarterly_stock = ai_holdings_ts.groupby(['QUARTER', 'TICKER'])['VALUE'].sum().reset_index()
quarterly_stock['RANK'] = quarterly_stock.groupby('QUARTER')['VALUE'].rank(ascending=False, method='first').astype(int)

# top 10 ranks per quarter
top10_per_quarter = quarterly_stock[quarterly_stock['RANK'] <= 10].pivot(
    index='RANK',
    columns='QUARTER', 
    values='TICKER'
)
top10_per_quarter

In [ ]:
# dollar values for top 10 per quarter
top10_values = quarterly_stock[quarterly_stock['RANK'] <= 10].pivot(
    index='RANK',
    columns='QUARTER',
    values='VALUE'
)
top10_values.style.format(thousands=',')

## Summary

This analysis demonstrates:
- Loading SEC 13F bulk data and matching to AI stocks
- Identifying AI stocks with the highest institutional holdings
- Aggregating by AI category (semiconductors, cloud, etc.)
- Calculating funds with the greatest AI stock holdings
- Tracking top stocks per quarter across the AI boom (2022q3+)

**Extensions:**
- Calculating AI stock value as percentage of total portfolio value
- Analyzing behavior around key events (e.g. ChatGPT launch, Gemini launch)
- Identifying funds increasing/decreasing AI exposure over time